<a href="https://colab.research.google.com/github/assansanogo/DL/blob/main/Dataset_preparation__Training_efficient_det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Build dataset as tfrecord (like Pascal VOC)

[Other tutorial on TFRecord](https://towardsdatascience.com/a-practical-guide-to-tfrecords-584536bc786c)

In [2]:
# clone tensorflow research folder 

In [1]:
!apt-get install nano

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  spell
The following NEW packages will be installed:
  nano
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 231 kB of archives.
After this operation, 778 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 nano amd64 2.9.3-2 [231 kB]
Fetched 231 kB in 1s (325 kB/s)
Selecting previously unselected package nano.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../nano_2.9.3-2_amd64.deb ...
Unpacking nano (2.9.3-2) ...
Setting up nano (2.9.3-2) ...
update-alternatives: using /bin/nano to provide /usr/bin/editor (editor) in auto mode
update-alternatives: using /bin/nano to provide /usr/bin/pico (pico) in auto mode
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 65305, done.
remote: Counting objects: 100% (721/721), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 65305 (delta 566), reused 700 (delta 553), pack-reused 64584
Receiving objects: 100% (65305/65305), 575.53 MiB | 29.07 MiB/s, done.
Resolving deltas: 100% (45690/45690), done.


In [3]:
cd /content/models/research

/content/models/research


In [4]:
# custom  csv file (from VGG annotator + parser tool)
!wget https://predicteev.s3.eu-west-3.amazonaws.com/computer_vision/csv_for_tf_record.csv

--2021-10-22 15:02:09--  https://predicteev.s3.eu-west-3.amazonaws.com/computer_vision/csv_for_tf_record.csv
Resolving predicteev.s3.eu-west-3.amazonaws.com (predicteev.s3.eu-west-3.amazonaws.com)... 52.95.155.48
Connecting to predicteev.s3.eu-west-3.amazonaws.com (predicteev.s3.eu-west-3.amazonaws.com)|52.95.155.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190273 (186K) [text/csv]
Saving to: ‘csv_for_tf_record.csv’

csv_for_tf_record.c 100%[===================>] 185.81K   610KB/s    in 0.3s    

2021-10-22 15:02:09 (610 KB/s) - ‘csv_for_tf_record.csv’ saved [190273/190273]



In [5]:
# download original images (labels are not necessary @ this point)
!wget https://predicteev.s3.eu-west-3.amazonaws.com/computer_vision/train.zip

--2021-10-22 15:02:12--  https://predicteev.s3.eu-west-3.amazonaws.com/computer_vision/train.zip
Resolving predicteev.s3.eu-west-3.amazonaws.com (predicteev.s3.eu-west-3.amazonaws.com)... 52.95.155.48
Connecting to predicteev.s3.eu-west-3.amazonaws.com (predicteev.s3.eu-west-3.amazonaws.com)|52.95.155.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330758999 (315M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>] 315.44M  28.5MB/s    in 12s     

2021-10-22 15:02:25 (26.5 MB/s) - ‘train.zip’ saved [330758999/330758999]



In [6]:
# unzip training observations in the current folder
!unzip ./train.zip 

Archive:  ./train.zip
   creating: train/
  inflating: __MACOSX/._train        
  inflating: train/MR TEMOWO OLUWATOBI   TELLERPAY APRIL STATEMENT_50.png  
  inflating: train/MR TEMOWO OLUWATOBI   TELLERPAY APRIL STATEMENT_44.png  
  inflating: train/Mr  festus oshokhue statement_16.png  
  inflating: train/LASISI STATEMENT_1.png  
  inflating: train/MR TEMOWO OLUWATOBI   TELLERPAY APRIL STATEMENT_78.png  
  inflating: train/MR TEMOWO OLUWATOBI   TELLERPAY APRIL STATEMENT_93.png  
  inflating: train/MR TEMOWO OLUWATOBI   TELLERPAY APRIL STATEMENT_87.png  
  inflating: train/420 PROPERTIES LIMITED_1.txt  
  inflating: train/MR TEMOWO OLUWATOBI   TELLERPAY APRIL STATEMENT_116.png  
  inflating: train/MR TAPIYA JOHN BANK STATEMENT_33.png  
  inflating: train/214773647637581511468199616204_96.txt  
  inflating: train/MR TEMOWO OLUWATOBI   TELLERPAY APRIL STATEMENT_102.png  
  inflating: train/MR TAPIYA JOHN BANK STATEMENT_27.png  
  inflating: train/MR SAMSON UGWANYI STATEMENT_143.txt  
  

In [7]:
!pip install tensorflow_model_optimization 
!pip install -U PyYAML

     |████████████████████████████████| 213 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 5.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
# Copyright 2021 predicteev-consulting
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""Convert raw labeled dataset to TFRecord for object_detection.
Example usage:
    python object_detection/dataset_tools/create_tf_record.py \
        --data_dir=/home/user/dataset \
        --year=2021 \
        --output_path=/home/user/dataset.record
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import io
import logging
import os

#from lxml import etree ( useful when using xml labels)
import PIL.Image
import tensorflow.compat.v1 as tf
import pandas as pd
import tqdm

# commodity code (will be probably re-written)
from object_detection.utils import dataset_util
import tensorflow_model_optimization as tfmot


In [9]:
flags = tf.app.flags
flags.DEFINE_string('data_dir', '', 'Root directory to raw PASCAL VOC dataset.')
flags.DEFINE_string('set', 'train', 'Convert training set, validation set or '
                    'merged set.')
flags.DEFINE_string('annotations_dir', 'Annotations',
                    '(Relative) path to annotations directory.')
flags.DEFINE_string('year', '2021', 'year of acquired data.')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('label_map_path', 'data/pascal_label_map.pbtxt',
                    'Path to label map proto')
FLAGS = flags.FLAGS

SETS = ['train', 'val', 'trainval', 'test']
YEARS = ['2021']


In [10]:
# When generating our tf record -format should be consistent (either PNG or JPG)
# Experimental (modified to work for a 1 class detection pb)
def csv_row_to_tf_example(data,idx,
                       format= "PNG"):
  
  """Convert csv labeled data to tf.Example proto.
    Notice that this function normalizes the bounding box coordinates provided
    by the raw data.
    Args:
      data: csv row from bigger abeled dataframe
      label_map_dict: A map from string label names to integers ids.
      ignore_difficult_instances: Whether to skip difficult instances in the
        dataset  (default: False).
      image_subdirectory: String specifying subdirectory holding the actual image data.
    Returns:
      example: The converted tf.Example.
    Raises:
      ValueError: if the image pointed to by data['filename'] is not a valid JPEG
    """

  full_path = data.loc[idx,"FULL_PATH"]
  width = int(data.loc[idx,'Width'])
  height = int(data.loc[idx,'Height'])

  with tf.gfile.GFile(full_path, 'rb') as fid:
    encoded_jpg = fid.read()
  

  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = PIL.Image.open(encoded_jpg_io)

  if image.format != format:
    raise ValueError('Image format not JPEG')

  key = hashlib.sha256(encoded_jpg).hexdigest()
  difficult = 0
  class_text = "table".encode()
  class_id = 1



  xmin = []
  ymin = []
  xmax = []
  ymax = []
  classes = []
  classes_text = []
  truncated = []
  poses = []
  difficult_obj = []


  xmin.append(data.loc[idx, 'X_min'] / width)
  ymin.append(data.loc[idx, 'Y_min'] / height)
  xmax.append(data.loc[idx, 'X_max'] / width)
  ymax.append(data.loc[idx, 'Y_max'] / height)
  difficult_obj.append(difficult)
  classes_text.append(class_text)
  classes.append(class_id)
  truncated.append(0)
  poses.append("No".encode())


  example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(
          data.loc[idx,'IMAGE_PATH'].encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(
          '0'.encode('utf8')),
      'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature('png'.encode('utf8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
      'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
      'image/object/truncated': dataset_util.int64_list_feature(truncated),
      'image/object/view': dataset_util.bytes_list_feature(poses),
  }))

  return example

In [11]:
# read the label file obtained with VGG labeling + parser tool
test_df = pd.read_csv("/content/models/research/csv_for_tf_record.csv")
# the path must be modified to work on a remote machine (hardcoded path)
test_df["FULL_PATH"]= test_df["FULL_PATH"].apply(lambda x: os.path.join( "/content/models/research/train", x.split("/")[-1] ))
test_df["IMAGE_PATH"]= test_df["FULL_PATH"].apply(lambda x:  x.split("/")[-1] )

In [12]:
# example of data parsed
test_df.head()

,REF_DIR,SUB_FOLDER,FULL_PATH,Width,Height,XC,YC,W,H,X_min,Y_min,X_max,Y_max,IMAGE_PATH
0,/Users/assansanogo/Downloads/to_label_for_LL,train,/content/models/research/train/420 PROPERTIES ...,1860,2300,890.5,1095.5,1730.0,1673.0,25.5,259.0,1755.5,1932.0,420 PROPERTIES LIMITED_1.png
1,/Users/assansanogo/Downloads/to_label_for_LL,train,/content/models/research/train/214773647637581...,2339,1653,1104.0,732.0,2068.0,1441.0,70.0,11.5,2138.0,1452.5,214773647637581511468199616204_96.png
2,/Users/assansanogo/Downloads/to_label_for_LL,train,/content/models/research/train/MR SAMSON UGWAN...,2339,1653,1100.5,730.5,2059.0,1433.0,71.0,14.0,2130.0,1447.0,MR SAMSON UGWANYI STATEMENT_143.png
3,/Users/assansanogo/Downloads/to_label_for_LL,train,/content/models/research/train/MR MOSES ADEYEM...,2339,1653,1104.0,733.5,2070.0,1441.0,69.0,13.0,2139.0,1454.0,MR MOSES ADEYEMI STATEMENT_2.png
4,/Users/assansanogo/Downloads/to_label_for_LL,train,/content/models/research/train/MR TAPIYA JOHN ...,2339,1653,1099.5,730.5,2057.0,1433.0,71.0,14.0,2128.0,1447.0,MR TAPIYA JOHN BANK STATEMENT_152.png


In [13]:
# Dataset tfrecord generation (for 1 example)
# will be looped through thanks to the idx

datum = test_df
csv_row_to_tf_example(datum, idx=0, format= "PNG")

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\211PNG\r\n\032\n\000\000\000\rIHDR\000\000\007D\000\000\010\374\010\002\000\000\000[<\316\031\000\001\000\000IDATx\234\354\335w\234\035Wy?\376\347\234\271\275o\221V\262d\331r\3056\006lb\007p\001\354P\022Z\022\0220\345KL\370\001\t_\010%\205\022J\200@H\276!@\200@ \204`Z\0148\024\307\241\004\343&c\014\330\006\033\027Y\335\352\333oo3\347\374\376x\356}tvn\321\356\312\262Y\351\363\306/\263{w\356\314\2313s\327\322\347>\3679\312ZK\000\000\000\000\000\000\000\000\000\000\360\353M?\332\003\000\000\000\000\000\000\000\000\000\000\200\303C\230\013\000\000\000\000\000\000\000\000\000\260\002 \314\005\000\000\000\000\000\000\000\000\000X\001\020\346\002\000\000\000\000\000\000\000\000\000\254\000\010s\001\000\000\000\000\000\000\000\000\000V\000\204\271\000\000\000\000\000\000\000\000\000\000+\000\302\\\000\000\000\000\000\000\000\000\000\200\025\000a.\000\000\000\000\000\000\000\000\000\300\n\2000\027\00

In [14]:
def create_dataset(datum, output_path):
  '''
  create tfrecord based on csv and fiven output_path
  '''
  writer = tf.python_io.TFRecordWriter(output_path)

  for idx in tqdm.tqdm(range(datum.shape[0])):
    tf_example = csv_row_to_tf_example(datum, idx,format= "PNG")
    writer.write(tf_example.SerializeToString())
    #print("file does not exist")
  writer.close()

In [15]:
ls ./

adversarial_text/      deeplab/                marco/             slim/
attention_ocr/         deep_speech/            nst_blogpost/      train/
audioset/              delf/                   object_detection/  train.zip
autoaugment/           efficient-hrl/          pcl_rl/            vid2depth/
cognitive_planning/    lfads/                  README.md
csv_for_tf_record.csv  lstm_object_detection/  rebar/
cvt_text/              __MACOSX/               seq_flow_lite/


In [16]:
cd /content

/content


In [17]:
!git clone https://github.com/google/automl.git

Cloning into 'automl'...
remote: Enumerating objects: 4011, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 4011 (delta 236), reused 300 (delta 188), pack-reused 3613
Receiving objects: 100% (4011/4011), 25.29 MiB | 22.52 MiB/s, done.
Resolving deltas: 100% (2988/2988), done.


In [18]:
ls /content

automl/  models/  sample_data/


In [19]:
cd automl/

/content/automl


In [20]:
ls .

CONTRIBUTING.md  efficientdet/  efficientnetv2/  LICENSE  README.md


In [21]:
cd /content/automl/efficientdet

/content/automl/efficientdet


In [22]:
# Dataset tfrecord generation (for all examples)
# will be looped through thanks to the idx
output_path = "OUT_TF2.record"
datum = test_df
create_dataset(datum, output_path)

100%|██████████| 835/835 [00:02<00:00, 292.79it/s]


In [23]:
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d0.tar.gz
!wget https://predicteev.s3.eu-west-3.amazonaws.com/computer_vision/config.yaml
!tar xf efficientdet-d0.tar.gz

--2021-10-22 15:03:02--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d0.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.159.128, 74.125.70.128, 74.125.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.159.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28994253 (28M) [application/octet-stream]
Saving to: ‘efficientdet-d0.tar.gz’

efficientdet-d0.tar 100%[===================>]  27.65M  78.3MB/s    in 0.4s    

2021-10-22 15:03:02 (78.3 MB/s) - ‘efficientdet-d0.tar.gz’ saved [28994253/28994253]

--2021-10-22 15:03:02--  https://predicteev.s3.eu-west-3.amazonaws.com/computer_vision/config.yaml
Resolving predicteev.s3.eu-west-3.amazonaws.com (predicteev.s3.eu-west-3.amazonaws.com)... 52.95.155.0
Connecting to predicteev.s3.eu-west-3.amazonaws.com (predicteev.s3.eu-west-3.amazonaws.com)|52.95.155.0|:443... connected.
HTTP request sent, awaiting response... 200 O

In [24]:
cd /content/automl/efficientdet

/content/automl/efficientdet


In [25]:
ls ./

aug/                        efficientdet-d0/        object_detection/
backbone/                   efficientdet-d0.tar.gz  OUT_TF2.record
coco_metric.py              g3doc/                  README.md
coco_metric_test.py         hparams_config.py       requirements.txt
config.yaml                 hparams_config_test.py  run_tflite.py
dataloader.py               inference.py            tensorrt.py
dataloader_test.py          __init__.py             testdata/
dataset/                    install_deps.sh         test.sh
Det-AdvProp.md              iou_utils.py            test_util.py
det_advprop_tutorial.ipynb  iou_utils_test.py       tf2/
det_model_fn.py             main.py                 tutorial.ipynb
det_model_fn_test.py        model_inspect.py        utils.py
efficientdet_arch.py        model_inspect_test.py   utils_test.py
efficientdet_arch_test.py   nms_np.py               visualize/


In [26]:
!ls /content/models/research/

adversarial_text       deeplab		      marco		slim
attention_ocr	       deep_speech	      nst_blogpost	train
audioset	       delf		      object_detection	train.zip
autoaugment	       efficient-hrl	      pcl_rl		vid2depth
cognitive_planning     lfads		      README.md
csv_for_tf_record.csv  lstm_object_detection  rebar
cvt_text	       __MACOSX		      seq_flow_lite


In [27]:
ls ./

aug/                        efficientdet-d0/        object_detection/
backbone/                   efficientdet-d0.tar.gz  OUT_TF2.record
coco_metric.py              g3doc/                  README.md
coco_metric_test.py         hparams_config.py       requirements.txt
config.yaml                 hparams_config_test.py  run_tflite.py
dataloader.py               inference.py            tensorrt.py
dataloader_test.py          __init__.py             testdata/
dataset/                    install_deps.sh         test.sh
Det-AdvProp.md              iou_utils.py            test_util.py
det_advprop_tutorial.ipynb  iou_utils_test.py       tf2/
det_model_fn.py             main.py                 tutorial.ipynb
det_model_fn_test.py        model_inspect.py        utils.py
efficientdet_arch.py        model_inspect_test.py   utils_test.py
efficientdet_arch_test.py   nms_np.py               visualize/


In [29]:
!python main.py --mode=train_and_eval\
    --train_file_pattern=/content/automl/efficientdet/OUT_TF2.record \
    --val_file_pattern=/content/automl/efficientdet/OUT_TF2.record \
    --model_name=efficientdet-d0 \
    --model_dir=/tmp/efficientdet-d0-finetune  \
    --ckpt=efficientdet-d0  \
    --train_batch_size=4 \
    --eval_batch_size=4 --eval_samples=12 \
    --num_examples_per_epoch=500 --num_epochs=10  \
    --hparams="num_classes=2"

2021-10-22 15:35:25.977145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 15:35:26.051321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 15:35:26.052313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
I1022 15:35:26.053863 140411370522496 main.py:228] {'name': 'efficientdet-d0', 'act_type': 'swish', 'image_size': (512, 512), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': None, 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 2, 'seg_num_classes': 3, 'hea

In [ ]:
 "/content/automl/efficientdet/OUT_TF2.record"

In [63]:
for example in tf.python_io.tf_record_iterator("/content/automl/efficientdet/OUT_TF2.record"):
    print(tf.train.Example.FromString(example))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
cd /

/


In [15]:
!git clone https://github.com/google/automl.git

Cloning into 'automl'...
remote: Enumerating objects: 4011, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 4011 (delta 237), reused 300 (delta 189), pack-reused 3613
Receiving objects: 100% (4011/4011), 25.29 MiB | 22.09 MiB/s, done.
Resolving deltas: 100% (2989/2989), done.


In [16]:
ls /automl/efficientdet/dataset/

create_coco_tfrecord.py       create_pascal_tfrecord_test.py  label_map_util.py
create_coco_tfrecord_test.py  __init__.py                     README.md
create_pascal_tfrecord.py     inspect_tfrecords.py            tfrecord_util.py


In [17]:
# Download and convert pascal data.
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar

--2021-10-22 13:48:48--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/x-tar]
Saving to: ‘VOCtrainval_11-May-2012.tar’

VOCtrainval_11-May- 100%[===================>]   1.86G  11.6MB/s    in 2m 53s  

2021-10-22 13:51:42 (11.0 MB/s) - ‘VOCtrainval_11-May-2012.tar’ saved [1999639040/1999639040]



In [147]:
!tar xf VOCtrainval_11-May-2012.tar

In [143]:
cd /automl/efficientdet

/automl/efficientdet


In [148]:

!mkdir tfrecord
!PYTHONPATH=".:$PYTHONPATH"  python /automl/efficientdet/dataset/create_pascal_tfrecord.py  \
    --data_dir=VOCdevkit --year=VOC2012  --output_path=tfrecord/pascal

mkdir: cannot create directory ‘tfrecord’: File exists
2021-10-22 12:00:29.500762: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-22 12:00:29.500829: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bae54d342570): /proc/driver/nvidia/version does not exist
I1022 12:00:29.501111 139705371359104 create_pascal_tfrecord.py:254] Writing to output directory: tfrecord
I1022 12:00:29.520346 139705371359104 create_pascal_tfrecord.py:287] Reading from PASCAL VOC2012 dataset.
I1022 12:00:29.520547 139705371359104 create_pascal_tfrecord.py:292] On image 0 of 5717
I1022 12:00:29.690605 139705371359104 create_pascal_tfrecord.py:292] On image 100 of 5717
I1022 12:00:29.828305 139705371359104 create_pascal_tfrecord.py:292] On image 200 of 5717
I1022 12:00:29.973217 139705371359104 create_pascal_tfrecord.py:292] On image 300 of 5717
I1022 12:

In [149]:
# Download backbone checkopints.
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d0.tar.gz
!tar xf efficientdet-d0.tar.gz

--2021-10-22 12:00:57--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d0.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28994253 (28M) [application/octet-stream]
Saving to: ‘efficientdet-d0.tar.gz’

efficientdet-d0.tar 100%[===================>]  27.65M  87.6MB/s    in 0.3s    

2021-10-22 12:00:58 (87.6 MB/s) - ‘efficientdet-d0.tar.gz’ saved [28994253/28994253]



In [150]:
ls

aug/                        g3doc/                  run_tflite.py
backbone/                   hparams_config.py       tensorrt.py
coco_metric.py              hparams_config_test.py  testdata/
coco_metric_test.py         inference.py            test.sh
dataloader.py               __init__.py             test_util.py
dataloader_test.py          install_deps.sh         tf2/
dataset/                    iou_utils.py            tfrecord/
Det-AdvProp.md              iou_utils_test.py       tutorial.ipynb
det_advprop_tutorial.ipynb  main.py                 utils.py
det_model_fn.py             model_inspect.py        utils_test.py
det_model_fn_test.py        model_inspect_test.py   visualize/
efficientdet_arch.py        nms_np.py               VOCdevkit/
efficientdet_arch_test.py   object_detection/       VOCtrainval_11-May-2012.tar
efficientdet-d0/            README.md
efficientdet-d0.tar.gz      requirements.txt


In [151]:
!wget https://predicteev.s3.eu-west-3.amazonaws.com/computer_vision/voc_config.yaml

--2021-10-22 12:02:28--  https://predicteev.s3.eu-west-3.amazonaws.com/computer_vision/voc_config.yaml
Resolving predicteev.s3.eu-west-3.amazonaws.com (predicteev.s3.eu-west-3.amazonaws.com)... 52.95.156.56
Connecting to predicteev.s3.eu-west-3.amazonaws.com (predicteev.s3.eu-west-3.amazonaws.com)|52.95.156.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 309 [binary/octet-stream]
Saving to: ‘voc_config.yaml’

voc_config.yaml     100%[===================>]     309  --.-KB/s    in 0s      

2021-10-22 12:02:28 (9.55 MB/s) - ‘voc_config.yaml’ saved [309/309]

